In [ ]:
%load_ext autoreload
%autoreload 2
from path_setup import setup_project_root
root = setup_project_root()

import os
import pandas as pd
# Local modules
from etl_showcase.infrastructure.utils.file_utils import (
    read_excel_sheets,
    save_large_dataframe_to_excel,
)

data_dirs_raw = os.path.join(os.getcwd(), 'data/raw')
data_dirs_preprocessed = os.path.join(os.getcwd(), 'data/processed')

# 1. 讀取資料並初步整理
print('Start to read data')
file_path = os.path.join(os.getcwd(), data_dirs_raw, 'Shopping_Data.xlsx')
target_sheet = 'Order_Items' 
dataset = read_excel_sheets(file_path = file_path, target_sheet = target_sheet)
# 如要了解訂單中同時出現A、B兩商品的頻率，即商品A出現1次和10次意義一樣，
# 保留訂單重複商品，會誇大支持度（Support）或信心度（Confidence），導致分析結果偏向容易被大量重複購買的消耗品。
# 如果是把 Product_ID 和 Quantity 各別記錄的資料格式，則可不用進行該操作。
df = dataset[['Order_ID', 'Product_ID']].drop_duplicates()

# 2. 預先計算基礎指標
print('Start to prepare basic variables')
# 計算總訂單數作為分母
total_orders = dataset['Order_ID'].nunique() 
# 計算每個產品單獨出現的次數 (用於計算 Lift)
product_counts = dataset.groupby('Product_ID').size().reset_index(name='Individual_Count')

# 3. 執行自我合併 (Self-Join)
print('Start to cross combine self')
merged = pd.merge(df, df, on='Order_ID', suffixes=('_A', '_B'))
print('Start to drop duplicate combine')
pair_df = merged[merged['Product_ID_A'] < merged['Product_ID_B']]

# 4. 群組化計算同時購買次數 (Pair_Count)
print('Start to group and create Pair_Count column')
result = pair_df.groupby(['Product_ID_A', 'Product_ID_B']).size().reset_index(name='Pair_Count')

# 5. 新增：計算 Support 與 Lift 
print('Start to calculate Support and Lift')

# 合併產品 A 的單獨出現次數
result = pd.merge(result, product_counts, left_on='Product_ID_A', right_on='Product_ID', how='left')
result = result.rename(columns={'Individual_Count': 'Count_A'}).drop('Product_ID', axis=1)

# 合併產品 B 的單獨出現次數
result = pd.merge(result, product_counts, left_on='Product_ID_B', right_on='Product_ID', how='left')
result = result.rename(columns={'Individual_Count': 'Count_B'}).drop('Product_ID', axis=1)

# [公式] Support(同時購買A 與 B比例) = 同時購買次數 / 總訂單數
result['Support'] = result['Pair_Count'] / total_orders

# [公式] Lift(推薦 A或B 之後，是否真的比隨機購買更有效) = P(A&B) / (P(A) * P(B))
# 換算後：(Pair_Count / total_orders) / ((Count_A/total_orders) * Count_B/total_orders)) = (Pair_Count * total_orders) / (Count_A * Count_B)
result['Lift'] = (result['Pair_Count'] * total_orders) / (result['Count_A'] * result['Count_B'])

# 依 Lift 降序排列，讓最有意義的規則排在最前面
result = result.sort_values(by='Lift', ascending=False).reset_index(drop=True)
# ----------------------------------

# 最後的 result 匯出成 excel
print('Start to export data')
data_file_path_preprocessed = os.path.join(os.getcwd(), data_dirs_preprocessed, 'Market_Basket_Analysis.xlsx')
save_large_dataframe_to_excel(result, data_file_path_preprocessed)

print('Process Completed!')